In [ ]:
import numpy as np
import GEOparse as gp
import pandas as pd
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib import rc
from IPython.display import display
import sklearn
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage
import seaborn
import combat
import math
from combat.pycombat import pycombat

We download the data from the GSE dataset

In [ ]:
gse1 = gp.get_GEO(geo="GSE157239", destdir="./")
gse2 = gp.get_GEO(geo="GSE147232", destdir="./")

In [ ]:
# DK, framework of code by JK
# Significance level
alpha2 = 0.01

# Identifying which samples are controls
control_columns2 = np.array([0, 1, 2, 3, 4])
c2 = np.zeros(10, dtype=bool)
c2[control_columns2] = 1

#print(gse2.gsms)
values2 = []
for _, v in gse2.gsms.items():
    ids2 = v.table["ID_REF"].to_numpy()
    values2.append(v.table['VALUE'].to_numpy())
values2 = np.stack(values2, axis=-1)

# Getting rid of the NaNs in the values array
nan_array = []
print(len(values2[:, 0]))
for i in np.arange(len(values2[:, 0])):
    for j in np.arange(len(values2[0, :])):
        if np.isnan(values2[i, j]):
            nan_array.append(i)

values2 = np.delete(values2, nan_array, axis = 0) 
ids2 = np.delete(ids2, nan_array, axis = 0)

#print(gse2.gpls['GPL18058'].table)
raw_data2 = gse2.gpls['GPL18058'].table
#print(raw_data2)
raw_data2 = raw_data2[pd.notna(raw_data2['Human_miRNA'])]
raw_ids2 = raw_data2['ID'].to_numpy()

mirna_idxs2 = [i for i, x in enumerate(ids2) if np.any(raw_ids2 == x)]
mirna_ids2 = ids2[mirna_idxs2]
values2 = values2[mirna_idxs2]

mask2 = np.broadcast_to(c2, (len(values2), 10))

controls2 = values2[mask2].reshape(-1, 5)
samples2 = values2[~mask2].reshape(-1, 5)

# Running one-way ANOVA analysis on the RNAs
f_stats2 = stats.f_oneway(controls2, samples2, axis=-1)
num_significant2 = np.sum(f_stats2.pvalue < alpha2)
signif_ids2 = mirna_ids2[f_stats2.pvalue < alpha2]

# Sorting by p-value
ids2 = np.array(ids2)
order2 = np.argsort(f_stats2.pvalue)
sorted_ids2 = ids2[order2]

sorted_pvalues2 = np.copy(f_stats2.pvalue)
sorted_pvalues2.sort()

print("number of samples: ", len(f_stats2.pvalue))

In [ ]:
# DK, framework of code by JK
signifs2 = []
mirna_signif_ids2 = []
for sig_id2 in signif_ids2:
    if not any(raw_data2["ID"] == sig_id2):
        continue
    signifs2.append(raw_data2[raw_data2["ID"] == sig_id2])
    mirna_signif_ids2.append(sig_id2)

signifs2 = pd.concat(signifs2)
mirna_signif_id_mask2 = [np.nonzero(mirna_ids2 == i)[0][0] for i in mirna_signif_ids2] #watch

signifs2.insert(0, "P-value", f_stats2.pvalue[mirna_signif_id_mask2])

signif_indices2 = []
for i in np.arange(len(f_stats2.pvalue)):
    if f_stats2.pvalue[i] < alpha2:
        signif_indices2.append(i)

# Plotting distribution of p-values
plt.hist(sorted_pvalues2, bins = 20, color = 'magenta', density = True)
plt.title('Distribution of p-values')
plt.xlabel('p-value')
plt.ylabel('Frequency')

#mirna_mask2 = signifs2['Sequence Type'] == 'miRNA'
#mirna_signifs2 = signifs2[mirna_mask]

In [ ]:
# DK, framework of code by JK
regulations2 = np.mean(samples2, axis=-1) > np.mean(controls2, axis=-1)

# True if upregulated, False if downregulated
signif_regulations2 = regulations2[mirna_signif_id_mask2]

labeled_sig_regs2 = np.stack([
    signifs2['Human_miRNA'].to_numpy(),
    signif_regulations2
], axis=-1)

print(labeled_sig_regs2)


In [ ]:
# DK, framework of code by JK
# Normalizing data for clustering
values_norm2 = values2

values_norm2 = values_norm2 - np.mean(values_norm2, axis=-1, keepdims=True)
values_norm2 = values_norm2 / np.std(values_norm2, axis=-1, keepdims=True)

In [ ]:
# DK, framework of code by JK
# Clustermap
dendro_df2 = pd.DataFrame(values_norm2[mirna_signif_id_mask2])
dendro_df2.columns = [('Control ' if i in control_columns2 else 'Sample ') + str(i) for i in range(10)]
dendro_df2.index = list(signifs2['Human_miRNA'])
display(seaborn.clustermap(dendro_df2, cmap='coolwarm'))

In [ ]:
# DK
# K-means clustering
sum_of_squared_distances2 = []
for k in range(1, 40):
    kmeans2 = KMeans(n_clusters = k, n_init = 20).fit(values_norm2)
    sum_of_squared_distances2.append(kmeans2.inertia_)
    
plt.plot(range(1, 40), sum_of_squared_distances2)
plt.xlabel('k')
plt.ylabel('Sum of Squared Distances')
plt.show()

In [ ]:
# DK
kmeans2 = KMeans(n_clusters = 8).fit(values_norm2)

#argmin, distance = sklearn.metrics.pairwise_distances_argmin_min(kmeans.cluster_centers_, values_norm)
#print(ids[argmin])

signif_labels_list2 = kmeans2.labels_[signif_indices2]
mirna_signif_labels_list2 = signif_labels_list2[0:38]

print(mirna_signif_labels_list2)

In [ ]:
# DK, framework of code by JK
signif_level2 = 0.1
order2 = np.argsort(kmeans2.labels_[f_stats2.pvalue < signif_level2])
clustered_values2 = values_norm2[f_stats2.pvalue < signif_level2][order2]
value_df2 = pd.DataFrame(clustered_values2)
value_df2.index = kmeans2.labels_[f_stats2.pvalue < signif_level2][order2]
value_df2.columns = [('Control ' if i in control_columns2 else 'Sample ') + str(i) for i in range(10)]
seaborn.clustermap(value_df2, cmap='mako', row_cluster=False, col_cluster=True)

In [ ]:
# JK
# Significance level
alpha = 0.01

# Identifying which samples are controls
control_columns = np.array([0, 1, 2, 4, 6, 9, 11, 15])
c = np.zeros(16, dtype=bool)
c[control_columns] = 1

# Creating accessible matrices from the dataset
values = []
for k, v in gse1.gsms.items():
    ids = v.table["ID_REF"].to_numpy()
    values.append(v.table['VALUE'].to_numpy())
values = np.stack(values, axis=-1)

raw_data = gse1.gpls['GPL21572'].table
raw_data = raw_data[raw_data['Sequence Type'] == 'miRNA']
raw_ids = raw_data['ID'].to_numpy()
mirna_idxs = [i for i, x in enumerate(ids) if np.any(raw_ids == x)]
mirna_ids = ids[mirna_idxs]
values = values[mirna_idxs]
print("Number of miRNA samples", values.shape[0])

mask = np.broadcast_to(c, (len(values), 16))

controls = values[mask].reshape(-1, 8)
samples = values[~mask].reshape(-1, 8)

# Running one-way ANOVA analysis on the RNAs
f_stats = stats.f_oneway(controls, samples, axis=-1)
num_significant = np.sum(f_stats.pvalue < alpha)
signif_ids = mirna_ids[f_stats.pvalue < alpha]

# Sorting by p-value
ids = np.array(ids)
order = np.argsort(f_stats.pvalue)
sorted_ids = ids[order]

In [ ]:
# JK
signifs = []
mirna_signif_ids = []
for sig_id in signif_ids:
    if not any(raw_data["ID"] == sig_id):
        continue
    signifs.append(raw_data[raw_data["ID"] == sig_id])
    mirna_signif_ids.append(sig_id)

signifs = pd.concat(signifs)
mirna_signif_id_mask = [np.nonzero(ids == i)[0][0] for i in mirna_signif_ids]
signifs.insert(17, "P-value", f_stats.pvalue[mirna_signif_id_mask])

sorted_pvalues = np.copy(f_stats.pvalue)
sorted_pvalues.sort()

signif_indices = []
for i in np.arange(len(f_stats.pvalue)):
    if f_stats.pvalue[i] < alpha:
        signif_indices.append(i)

# Plotting distribution of p-values
plt.hist(sorted_pvalues, bins = 20, color = 'gray')
plt.title('Distribution of p-values')
plt.xlabel('p-value')
plt.ylabel('Frequency')

mirna_mask = signifs['Sequence Type'] == 'miRNA'
mirna_signifs = signifs[mirna_mask]

To determine if each gene is upregulated or downregulated, we compare the mean values against the control.

In [ ]:
# JK
regulations = np.mean(samples, axis=-1) > np.mean(controls, axis=-1)

# True if upregulated, False if downregulated
signif_regulations = regulations[mirna_signif_id_mask]

labeled_sig_regs = np.stack([
    signifs['Transcript ID(Array Design)'].to_numpy(),
    signif_regulations
], axis=-1)

print(labeled_sig_regs)

In [ ]:
# DK
# Creating table of p-values, regulation patterns, and functions of each miRNA of interest
data = {'P-value': signifs['P-value'].values, 
        'Regulation (case vs. control)':['Upregulated', 'Downregulated', 'Upregulated', 'Downregulated', 'Downregulated', 'Upregulated', 'Downregulated', 'Downregulated', 'Downregulated', 'Upregulated', 
                                         'Upregulated', 'Upregulated', 'Upregulated', 'Upregulated', 'Upregulated', 'Downregulated', 'Downregulated', 'Downregulated', 'Downregulated', 'Downregulated'], 
        'Function':['Tumor suppression$^{13}$', 'Cell proliferation$^{14}$', 'Tumor suppression$^{15}$', 'Tumorigenesis$^{16}$', 'Regulates A\u03B2$^{17}$', 'Cell proliferation$^{18}$', 'Cell proliferation$^{19}$', 'Cell proliferation$^{20}$',
                   'Tumor suppression$^{21}$', 'Tumor suppression$^{22}$', 'Lipid metabolism$^{23}$', 'N/A', 'N/A', 'N/A', 'Tumor suppression$^{24}$', 'N/A', 'N/A', 'N/A', 'N/A', 'Regulate autoimmune disease$^{25}$']} 


df = pd.DataFrame(data, index =['miR-215-3p', 'miR-369-5p', 'miR-429', 'miR-767-5p', 'miR-1251-5p', 'miR-1470', 'miR-3180-5p', 'miR-4286', 'miR-500b-3p', 'miR-3912-5p', 'miR-3929', 'miR-4540', 'miR-4633-3p',
                               'miR-4653-5p', 'miR-203b-5p', 'miR-4791', 'miR-5003-3p', 'miR-5093', 'miR-6877-5p', 'miR-7155-3p']) 

display(df)


In [ ]:
# JK
# Normalizing data for clustering
values_norm = values

values_norm = values_norm - np.mean(values_norm, axis=-1, keepdims=True)
values_norm = values_norm / np.std(values_norm, axis=-1, keepdims=True)

In [ ]:
# DK
# K-means clustering
sum_of_squared_distances = []
for k in range(1, 40):
    kmeans = KMeans(n_clusters = k, n_init = 20).fit(values_norm)
    sum_of_squared_distances.append(kmeans.inertia_)
    
plt.plot(range(1, 40), sum_of_squared_distances, color = 'black')
plt.xlabel('k')
plt.ylabel('Sum of Squared Distances')
plt.show()


In [ ]:
# DK
kmeans = KMeans(n_clusters = 8, n_init = 20).fit(values_norm)

#argmin, distance = sklearn.metrics.pairwise_distances_argmin_min(kmeans.cluster_centers_, values_norm)
#print(ids[argmin])

signif_labels_list = kmeans.labels_[signif_indices]
mirna_signif_labels_list = signif_labels_list[0:20]

print(mirna_signif_labels_list)

In [ ]:
# JK
# Here we analyze the functions within each cluster above
unknown_idxs = [mirna_signif_id_mask[i] for i in range(len(mirna_signif_id_mask)) if list(df['Function'])[i] == 'N/A']
unknown_clusters = [mirna_signif_labels_list[i] for i in range(len(mirna_signif_id_mask)) if list(df['Function'])[i] == 'N/A']

num_nearest = 3

for unknown_idx, cluster in zip(unknown_idxs, unknown_clusters):
    print(f"Nearest items in cluster {cluster} with {raw_data['Transcript ID(Array Design)'][raw_data['ID'] == mirna_ids[unknown_idx]].item()}")
    in_cluster = (kmeans.labels_ == cluster)
    cluster_center = kmeans.cluster_centers_[cluster]
    
    distances_to_center = values_norm[in_cluster] @ cluster_center
    closest = np.argsort(distances_to_center)[-num_nearest:]
    near_ids = mirna_ids[in_cluster][closest]
    near = [raw_data[raw_data['ID'] == near_id] for near_id in near_ids]
    display(pd.concat(near))



| Unknown miRNA | Cluster Centers | Cluster Center Functions |
| --- | --- | --- |
| miR-4540<br>miR-4633-3p<br>miR-4653-5p  | miR-3714<br>miR-5096<br>miR-4755-3p | N/A<br>Promotes cancer$^{26}$<br>N/A |
| miR-4791<br>miR-5003-3p<br>miR-5093 | miR-6782-3p<br>miR-4313<br>miR-200c-5p | N/A<br>N/A<br>Protects against<br> Alzheimer's$^{27}$ |
|miR-6877-5p | miR-96-5p<br>miR-581<br>miR-4803 | N/A<br>N/A<br>N/A |

In [ ]:
# JK
signif_level = 0.01
order = np.argsort(kmeans.labels_[f_stats.pvalue < signif_level])
clustered_values = values_norm[f_stats.pvalue < signif_level][order]
value_df = pd.DataFrame(clustered_values)
value_df.index = kmeans.labels_[f_stats.pvalue < signif_level][order]
value_df.columns = [('Control ' if i in control_columns else 'Sample ') + str(i) for i in range(16)]
seaborn.clustermap(value_df, cmap='mako', row_cluster=False, col_cluster=True)


In [ ]:
# DK (not used in final paper)
# PCA
pca = PCA(n_components = 16)
pca.fit(values_norm)
print(pca.explained_variance_ratio_)

In [ ]:
# JK
clustering = linkage(values_norm[mirna_signif_id_mask])
fig = plt.figure()

dendrogram(
    clustering,
    labels = list(signifs['Transcript ID(Array Design)']),
    leaf_rotation=90,
)

plt.show()

In [ ]:
# JK
dendro_df = pd.DataFrame(values_norm[mirna_signif_id_mask])
control_columns = np.array([0, 1, 2, 4, 6, 9, 11, 15])
labels = [''] * 16
c_n = 1
s_n = 1
for i in range(16):
    if i in control_columns:
        labels[i] = 'Control ' + str(c_n)
        c_n += 1
    else: 
        labels[i] = 'Case ' + str(s_n)
        s_n += 1
    
dendro_df.columns = [labels]
dendro_df.index = list(signifs['Transcript ID(Array Design)'])
hmap1 = seaborn.clustermap(dendro_df, cmap='coolwarm')
ax = hmap1.ax_heatmap
ax.set_xlabel('Samples')

display(hmap1)

In [ ]:
# DK
# COMBAT

gene_names_1 = [raw_data[raw_data['ID'] == id]['Transcript ID(Array Design)'].item() for id in mirna_ids]

gene_names_2 = [raw_data2[raw_data2['ID'] == id]['Human_miRNA'].item() for id in mirna_ids2]

combat_1 = pd.DataFrame(data = values)
combat_1.columns = {'GSM4759790', 'GSM4759791', 'GSM4759792', 'GSM4759793', 
                     'GSM4759794', 'GSM4759795', 'GSM4759796', 'GSM4759797', 'GSM4759798', 'GSM4759799', 
                    'GSM4759800', 'GSM4759801', 'GSM4759802', 'GSM4759803', 'GSM4759804', 'GSM4759805'}
combat_1.insert(0, "miRNA Name", gene_names_1)

combat_2 = pd.DataFrame(data = values2)
combat_2.columns = {'GSM4421278', 'GSM4421279', 'GSM4421280', 'GSM4421281', 'GSM4421282', 
                   'GSM4421283', 'GSM4421284', 'GSM4421285', 'GSM4421286', 'GSM4421287', }
combat_2.insert(0, "miRNA Name", gene_names_2)


df_expression = combat_1.merge(combat_2, on="miRNA Name")

final_genes = df_expression['miRNA Name'].tolist()

df_expression.drop('miRNA Name', axis = 1, inplace = True)
df_expression.index = final_genes

batch = [0] * 16 + [1] * 10

df_corrected = pycombat(df_expression, batch)

print(len(df_corrected))

In [ ]:
# DK, framework of code by JK
# Significance level
alpha3 = 0.01

# Identifying which samples are controls
control_columns3 = np.array([0, 1, 3, 9, 10, 11, 12, 13, 16, 18, 19, 23, 24])
c3 = np.zeros(26, dtype=bool)
c3[control_columns3] = 1

# Creating accessible matrices from the dataset
mask = np.broadcast_to(c3, (len(df_corrected), 26))

df_corrected_array = df_corrected.to_numpy()

controls3 = df_corrected_array[mask].reshape(-1, 13)
samples3 = df_corrected_array[~mask].reshape(-1, 13)

print(controls3.shape)

# Running one-way ANOVA analysis on the RNAs
f_stats3 = stats.f_oneway(controls3, samples3, axis=-1)

signif_pvalue = []
for i in np.arange(len(f_stats3.pvalue)):
    if f_stats3.pvalue[i] < alpha3:
        signif_pvalue.append(i)

        
print(signif_pvalue)
final_genes = np.array(final_genes)
num_significant3 = np.sum(f_stats3.pvalue < alpha3)
signif_ids3 = final_genes[signif_pvalue]

print(signif_ids3)

In [ ]:
# DK, framework of code by JK
signifs3 = []
mirna_signif_ids3 = []
print(signif_ids3)
for sig_id in signif_ids3:
    if not any(final_genes == sig_id):
        continue
    signifs3.append(np.where(final_genes == sig_id)[0][0])

sorted_pvalues3 = np.copy(f_stats3.pvalue)
sorted_pvalues3.sort()

signifs3 = np.array(signifs3)

signif_indices3 = []
for i in np.arange(len(f_stats3.pvalue)):
    if f_stats3.pvalue[i] < alpha3:
        signif_indices3.append(i)

# Plotting distribution of p-values
plt.hist(sorted_pvalues3, bins = 20, color = 'gray')
plt.title('Distribution of p-values')
plt.xlabel('p-value')
plt.ylabel('Frequency')



In [ ]:
# DK
# Creating table of p-values, regulation patterns, and functions of each miRNA of interest
data = {'P-value': f_stats3.pvalue[signif_indices3], 
        'Regulation (case vs. control)':['Upregulated', 'Upregulated', 'Downregulated'], 
        'Function': ['N/A', 'Downregulated in cancer$^{28}$', 'N/A']} 


df = pd.DataFrame(data, index =['miR-2681-3p', 'miR-5681a', 'miR-5684']) 

display(df)

In [ ]:
# DK, framework of code by JK
regulations3 = np.mean(samples3, axis=-1) > np.mean(controls3, axis=-1)

# True if upregulated, False if downregulated
signif_regulations3 = regulations3[signifs3]

labeled_sig_regs3 = np.stack([
    final_genes[signifs3],
    signif_regulations3
], axis=-1)

print(labeled_sig_regs3)

In [ ]:
# JK
# Normalizing data for clustering
values_norm3 = df_corrected_array

values_norm3 = values_norm3 - np.mean(values_norm3, axis=-1, keepdims=True)
values_norm3 = values_norm3 / np.std(values_norm3, axis=-1, keepdims=True)

print(values_norm3.shape)

In [ ]:
# DK
# K-means clustering
sum_of_squared_distances3 = []
for k in range(1, 40):
    kmeans3 = KMeans(n_clusters = k).fit(values_norm3)
    sum_of_squared_distances3.append(kmeans3.inertia_)
    
plt.plot(range(1, 40), sum_of_squared_distances3, color = 'black')
plt.xlabel('k')
plt.ylabel('Sum of Squared Distances')
plt.show()

In [ ]:
# DK
kmeans3 = KMeans(n_clusters = 8).fit(values_norm3)

signif_level3 = 0.05
order3 = np.argsort(kmeans3.labels_[f_stats3.pvalue < signif_level3])
clustered_values3 = values_norm3[f_stats3.pvalue < signif_level3][order3]
value_df3 = pd.DataFrame(clustered_values3)
value_df3.index = kmeans3.labels_[f_stats3.pvalue < signif_level3][order3]

control_columns3 = np.array([0, 1, 3, 9, 10, 11, 12, 13, 16, 18, 19, 23, 24])
labels3 = [''] * 26
c_n = 1
s_n = 1
for i in range(26):
    if i in control_columns3:
        labels3[i] = 'Control ' + str(c_n)
        c_n += 1
    else: 
        labels3[i] = 'Case ' + str(s_n)
        s_n += 1

value_df3.columns = [labels3]

hmap3 = seaborn.clustermap(value_df3, row_cluster = False, col_cluster=True)
ax = hmap1.ax_heatmap
ax.set_xlabel('Samples')

display(hmap3)

In [ ]:
# DK, framework of code by JK
dendro_df3 = pd.DataFrame(values_norm3)
dendro_df3.columns = [('Control ' if i in control_columns3 else 'Sample ') + str(i) for i in range(26)]
dendro_df3.index = list(final_genes)
display(seaborn.clustermap(dendro_df3, cmap='coolwarm'))